# Development Notebook of Benji

...basically a nice way to run my code ;) In its own notebook for version management reasons.

In [ ]:
import threading
from queue import Queue
import cv2

from src.camera_capture import FrameContainer
from src.camera_capture import CameraSetup
from src.camera_capture import capture_frames
from src.camera_capture import get_frame_from_queue

### PARAMETERS ###

keyword = "bottle"
"""Keyword inputed by user = what robot needs to find"""

### FUNCTIONS ###

"""
Note

This block is to be removed eventually. Functions and classes should all live in files.
"""

def process_frames(window_name : str, queue_frame: Queue, event_stop: threading.Event):
    while True:
        frame_container = get_frame_from_queue(queue_frame)
        if frame_container is None:
            continue
        cv2.imshow(window_name, frame_container.get_info_frame())
        if cv2.waitKey(1) & 0xFF == ord('q'): #stop if the q button is pressed
            event_stop.set()
            return


### SCRIPT ###
print(f"Searching for '{keyword}'...")

# set up camera capturing
window_name = "Raw Capturing"
camera_setup = CameraSetup(0,1) #capturing camera port 0 and camera port 1
queue_frame = Queue(maxsize=5) #capturing to a queue of length 5
event_stop = threading.Event()

thread_capture = threading.Thread( #setting up thread
    target=capture_frames,
    args=(camera_setup, queue_frame, event_stop)
)
thread_capture.start()
print("Started camera capturing.")

# process the frames

try:
    #process the captured frames
    process_frames(window_name=window_name, queue_frame=queue_frame, event_stop=event_stop)
finally:
    # Stop capture thread
    event_stop.set()
    thread_capture.join()
    cv2.destroyAllWindows()